In [ ]:
# !pip install stanza
# !pip install ollama
# !pip install vertexai
# !pip install -t lib google-auth google-auth-httplib2 google-api-python-client --upgrade
# !pip install pandas_gbq
# !pip install python-dotenv

In [1]:
import pandas as pd
import os
import re
import stanza
import random
import logging
import pymorphy3
import pandas as pd
import json

# from langchain_openai import ChatOpenAI
# from langchain import PromptTemplate

from tqdm import tqdm

tqdm.pandas()

/Users/linndfors/study/diploma/new_venv/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


## READ ann files

In [25]:
def find_ann_files(directory):
    ann_files = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".ann"):
                file_path = os.path.join(root, file)
                ann_files.append(file_path)

    return ann_files

In [90]:
def read_ann_files(directory):
    ann_files = find_ann_files(directory)
    job_title_sentences = []

    for ann_file in ann_files:
        with open(ann_file, "r", encoding="utf-8") as ann_f:
            for ann_l in ann_f:
                ann_row = ann_l.split("\t")

                try:
                    if ann_row[1] == "JOB":
                        job_title_sentences.append(ann_row)
                except Exception as e:
                    continue

    return job_title_sentences

In [6]:
job_title_entities_ng = read_ann_files("../ner_for_fem/data/ng")

In [7]:
job_title_entities_ng

[['T21', 'JOB', '800', '820', 'заступника директора\n'],
 ['T31', 'JOB', '1071', '1079', 'депутата\n'],
 ['T36', 'JOB', '1180', '1198', 'радником директора\n'],
 ['T42', 'JOB', '1373', '1383', 'екс-нардеп\n'],
 ['T44', 'JOB', '1406', '1417', 'екс-власник\n'],
 ['T55', 'JOB', '1701', '1714', 'екс-міністрів\n'],
 ['T58', 'JOB', '1756', '1774', 'народного депутата\n'],
 ['T21', 'JOB', '955', '970', 'співзасновником\n'],
 ['T23', 'JOB', '1000', '1012', 'замкерівника\n'],
 ['T29', 'JOB', '1177', '1193', 'голови правління\n'],
 ['T32', 'JOB', '1252', '1264', 'екс-депутата\n'],
 ['T30', 'JOB', '1692', '1732', 'начальником виробничо-технічного відділу\n'],
 ['T1', 'JOB', '9', '17', 'очільник\n'],
 ['T11', 'JOB', '343', '355', 'Екс-керівник\n'],
 ['T19', 'JOB', '750', '756', 'голови\n'],
 ['T29', 'JOB', '963', '976', 'екс-очільника\n'],
 ['T37', 'JOB', '1323', '1332', 'чиновника\n'],
 ['T41', 'JOB', '1502', '1513', 'засновників\n'],
 ['T21', 'JOB', '846', '850', 'мера\n'],
 ['T27', 'JOB', '991'

## NER

In [44]:
import os
import numpy as np
import spacy
import tqdm
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [56]:
nlp = spacy.load("uk_ner_web_trf_13class")

/Users/linndfors/study/diploma/new_venv/lib/python3.10/site-packages/spacy_transformers/layers/hf_shim.py:137: UserWarning: Error loading saved torch state_dict with strict=True, likely due to differences between 'transformers' versions. Attempting to load with strict=False as a fallback...

If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current 'transformers' and 'spacy-transformers' versions. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [3]:
def find_txt_files(directory):
    txt_files = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                txt_files.append(file_path)

    return txt_files

In [4]:
def read_txt_files(directory):
    txt_files = find_txt_files(directory)
    text = []

    for txt_file in txt_files:
        with open(txt_file, "r", encoding="utf-8") as txt_f:
            text.append(txt_f.read())

    return text

In [12]:
text_ng = read_txt_files("/Users/linndfors/study/diploma/ner_for_fem/data/ng")

In [13]:
print("кількість текстів ng:", len(text_ng))
print("кількість чарів ng :", len("".join(text_ng)))

# number_of_chars_for_text = len("".join(text_ng))/len(text_ng)
# print(number_of_chars_for_text)

кількість текстів ng: 298
кількість чарів ng : 572679


In [ ]:
# total_number_of_sentences = 0


# for x in text_ng:
#     sentences = [s for s in x.split('\n') if s.strip()]
#     total_number_of_sentences += len(sentences)

In [ ]:
# avarage_number_of_sent = (total_number_of_sentences/len(text_ng))
# print(avarage_number_of_sent)

15.023489932885907


In [ ]:
# average_number_of_tokens_in_sentence_ng = (number_of_chars_for_text/avarage_number_of_sent)/4
# print(average_number_of_tokens_in_sentence_ng)

31.978947956220683


In [14]:
text_bruk = read_txt_files("/Users/linndfors/study/diploma/ner_for_fem/data/bruk")

In [ ]:
# total_number_of_sentences_bruk = 0
# total_chars = 0

# for x in text_bruk:
#     sentences = [s for s in x.split('\n') if s.strip()]
#     total_number_of_sentences_bruk += len(sentences)
#     for sent in sentences:
#         total_chars += len(sent)

In [16]:
print("texts bruk:", len(text_bruk))
print("chars bruk:", len("".join(text_bruk)))

texts bruk: 262
chars bruk: 1346679


In [ ]:
# number_of_chars_for_text_bruk = len("".join(text_bruk))/len(text_bruk)
# print(number_of_chars_for_text_bruk)

5139.996183206107


In [ ]:
# avarage_number_of_sent_bruk = (total_number_of_sentences_bruk/len(text_bruk))
# print(avarage_number_of_sent_bruk)

52.908396946564885


In [ ]:
# average_number_of_tokens_in_sentence_bruk = (number_of_chars_for_text_bruk/avarage_number_of_sent_bruk)/4
# print(average_number_of_tokens_in_sentence_bruk)

24.28724210070697


### Get sentences with JOB TITLEs

In [19]:
def parse_ann_file(ann_file):
    entities = []

    with open(ann_file, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split("\t")
            if len(parts) < 3:
                continue
            label = parts[1]
            start = parts[2]
            end = parts[3]
            text = parts[4]

            entities.append({"text": text, "start": start, "end": end, "label": label})

    return entities


def extract_entities(txt_file, ann_file):
    labeled_sentences_list = []
    sentences = []

    with open(txt_file, "r", encoding="utf-8") as f:
        sentence_lines = f.readlines()

    entities = parse_ann_file(ann_file)

    for sentence in sentence_lines:
        labeled_dict = {}
        job = False
        sentence = sentence.strip()

        for ent in entities:
            if ent["text"] in sentence:
                labeled_dict[ent["text"]] = ent["label"]
                if ent["label"] == "JOB":
                    job = True

        if job:
            labeled_sentences_list.append(labeled_dict)
            sentences.append(sentence)

    return labeled_sentences_list, sentences

In [18]:
def get_sentence_with_job_titles(ann_files):

    sentence_with_jobs = {}

    for ann_file in ann_files:
        txt_file = ann_file.replace(".ann", ".txt")
        labeled_sentences_list, sentences = extract_entities(txt_file, ann_file)
        for i in range(len(sentences)):
            sentence_with_jobs[sentences[i]] = labeled_sentences_list[i]

    return sentence_with_jobs

In [66]:
ng_ann_files = find_ann_files("/Users/linndfors/study/diploma/ner_for_fem/data/ng")

ng_sentence_with_jobs = get_sentence_with_job_titles(ng_ann_files)

In [67]:
len(ng_sentence_with_jobs.keys())

1029

In [68]:
bruk_ann_files = find_ann_files("/Users/linndfors/study/diploma/ner_for_fem/data/bruk")

bruk_sentence_with_jobs = get_sentence_with_job_titles(bruk_ann_files)

In [69]:
len(bruk_sentence_with_jobs.keys())

560

# LLMs

## Llama

In [81]:
# import ollama

# options = {
#     "num_predict": 80,
#     "temperature": 0
# }

# def swap_gender_2(word):
#     response = ollama.generate(model='llama2',
#                             prompt=
#                             '''Зміни гендер слова "{word}".
#                             Поверни змінене слово українською мовою у форматі JSON.
#                             Ти маєш зберігати відмінок слова, наприклад, якщо слово було в родовому відмінку "режисера", то і слово зі зміненим гендером має бути в родовому відмінку "режисерки".
#                             Ти маєш повертати форму множини, якщо слово було в множині ("власники": "власниці") або повертати форму однини, якщо оригінальне слово мало форму однини.
#                             Твоя відповідь має бути у такому форматі: {{"{word}": "<змінене_слово>"}} без зайвих деталей і пояснень'''
#                             .format(word=word),
#                             options=options)
#     return response

In [85]:
options = {"num_predict": 80, "temperature": 0}


def swap_gender(word):
    response = ollama.generate(
        model="mistral",
        prompt="""Зміни гендер слова "{word}".
                            Поверни змінене слово українською мовою у форматі JSON.
                            Ти маєш зберігати відмінок слова, наприклад, якщо слово було в родовому відмінку "режисера", то і слово зі зміненим гендером має бути в родовому відмінку "режисерки".
                            Ти маєш повертати форму множини, якщо слово було в множині ("власники": "власниці") або повертати форму однини, якщо оригінальне слово мало форму однини.
                            Якщо ти не знаєш як утворити слова з протилежним гендером, то залиш оригінальне слово на місці зміненого.
                            Твоя відповідь має бути у такому форматі: {{"{word}": "<змінене_слово>"}} без зайвих деталей і пояснень""".format(
            word=word
        ),
        options=options,
    )
    return response

In [78]:
word_to_swap = "художника"
response = swap_gender_2(word_to_swap)

In [72]:
swapped_word = json.loads(response["response"])[word_to_swap]

print(swapped_word)

математикиня


read words

In [48]:
with open("job_titles/job_title_entities_bruk.txt", "r") as file:
    job_title_entities_bruk = [line.strip() for line in file]

print(job_title_entities_bruk)

['заступник', 'художника', 'лектори', 'режисером', 'народних депутатів', 'видавцями', 'голова', 'голова', 'тренери', 'науковець', 'фотографа', 'тележурналіст', 'викладачі-методисти', 'Доцент', 'викладачі', 'викладач', 'викладач', 'журналісти', 'священиком', 'священика', 'священика', 'співробітників', 'мол . наук . співробітника', 'програміста', 'редактор', 'пекар', 'охоронці', 'актриса', 'начальника', 'музейниками', 'священик', 'священик', 'панотця', 'кобзарі', 'Орнітологи', 'орнітологи', 'письменник', 'начальникові', 'геолог', 'військовим', 'позаштатний кореспондент', 'священиків', 'режисера', 'князь', 'поет-символіст', 'поет', 'поет', 'байкар', 'прозаїк', 'співець', 'поетів', 'письменники', 'психологів', 'дослідники', 'психологи', 'психологи', 'психолог', 'поетів', 'поетів', 'художник', 'психіатр', 'водія', 'доктор', 'голова', 'губернатор', 'нардеп', 'голові', 'депутатом', 'голови', 'керівник', 'голова', 'голову', 'офіцер', 'портьє', 'адміністратора', 'поет', 'драматург', 'письменник

In [49]:
len(job_title_entities_bruk)

638

In [92]:
len(set(job_title_entities_bruk))

437

In [94]:
test_job_title_entities_bruk = set(job_title_entities_bruk)
print(test_job_title_entities_bruk)

{'поет', 'президента', 'пастир', 'наукові редакторки', 'народний депутат', 'культурологи', 'ктиторів', 'керівник', 'пастирів', 'військовослужбовець', 'професорка', 'театральний режисер', 'ченцям', 'помічник інспектора студентів', 'учителі', 'майстру спорту', 'поетом', 'шафарям', 'наркомом', 'стрільці', 'інспектор', 'судді', 'учителі-ченці', 'митрополиту', 'режисери-постановники', 'єпископи', 'спортсменок', 'депутатом', 'лектори', 'ШОКОЛАДНИЦЯ', 'митрополита', 'художників', 'тележурналіст', 'Слідчий', 'кутюр’є', 'лікаря', 'Головнокомандувача Збройних Сил України', 'курінний', 'ченців', 'підприємець', 'Міністра оборони України', 'письменник', 'коронні гетьмани', 'Поетеса', 'головою', 'політолог', 'логопеда', 'беркутівця', 'Художники', 'служка', 'кошового отамана', 'служницю', 'істориків', 'продавчиня', 'панотця', 'заступник міністра', 'візажистів', 'чиновниками', 'садівників', 'кандидата філологічних наук', 'поетеса', 'портьє', 'мовознавиця', 'редактори', 'депутати', 'ченця', 'отаман', '

In [102]:
swapped_words_result = {}

for job_title in test_job_title_entities_bruk:
    try:
        response = swap_gender(job_title)
        response_content = response["response"]
        print(response_content)
        swapped_word = list(json.loads(response_content).values())[0]
        print(f"{job_title}: {swapped_word}")
        swapped_words_result[job_title] = swapped_word
    except Exception as e:
        print("error:", e)
        print("job_title:", job_title)

{"поет": "поетеса"}
поет: поетеса
{"президента": "президентка"}
президента: президентка
{"пастир": "пастирка"}
пастир: пастирка
{"наукові редакторки": "наукові редактори"}
наукові редакторки: наукові редактори
{"народний депутат": "народна депутатка"}
народний депутат: народна депутатка
{"культурологи": "культурологині"}
культурологи: культурологині
{"ктиторів": "ктиторок"}
ктиторів: ктиторок
{"керівник": "керівниця"}
керівник: керівниця
{"пастирів": "пастирки"}
пастирів: пастирки
{"військовослужбовець": "військовослужбовиця"}
військовослужбовець: військовослужбовиця
{"професорка": "професор"}
професорка: професор
{"театральний режисер": "театральна режисерка"}
театральний режисер: театральна режисерка
{"ченцям": "ченців"}
ченцям: ченців
{"помічник інспектора студентів": "помічниця інспектора студентів"}
помічник інспектора студентів: помічниця інспектора студентів
{"учителі": "учительки"}
учителі: учительки
{"майстра спорту": "майстрині спорту"}
майстру спорту: майстрині спорту
{"поет

KeyboardInterrupt: 

In [103]:
swapped_words_result

{'поет': 'поетеса',
 'президента': 'президентка',
 'пастир': 'пастирка',
 'наукові редакторки': 'наукові редактори',
 'народний депутат': 'народна депутатка',
 'культурологи': 'культурологині',
 'ктиторів': 'ктиторок',
 'керівник': 'керівниця',
 'пастирів': 'пастирки',
 'військовослужбовець': 'військовослужбовиця',
 'професорка': 'професор',
 'театральний режисер': 'театральна режисерка',
 'ченцям': 'ченців',
 'помічник інспектора студентів': 'помічниця інспектора студентів',
 'учителі': 'учительки',
 'майстру спорту': 'майстрині спорту',
 'поетом': 'поетеса',
 'шафарям': 'шафарки',
 'наркомом': 'наркоминка',
 'стрільці': 'стрільчині',
 'інспектор': 'інспекторка',
 'судді': 'суддя',
 'учителі-ченці': 'учительки',
 'митрополиту': 'митрополичий',
 'режисери-постановники': 'режисерки',
 'єпископи': 'єпископки',
 'спортсменок': 'спортсменки',
 'депутатом': 'депутатка',
 'лектори': 'лекторки',
 'митрополита': 'митрополичка',
 'художників': 'художниць',
 'тележурналіст': 'тележурналістка',
 

## Gemini

In [ ]:
from google.oauth2 import service_account
import pandas_gbq
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models
from dotenv import load_dotenv

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = ("key")

GOOGLE_APPLICATION_CREDENTIALS = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

# auth gcp
credentials = service_account.Credentials.from_service_account_file(
    GOOGLE_APPLICATION_CREDENTIALS
)
pandas_gbq.context.credentials = credentials
project_name = os.getenv("project")
vertexai.init(project=os.getenv("project"), location="europe-central2")

In [106]:
generation_config = {
    "max_output_tokens": 1000,
    "temperature": 0.95,
    "top_p": 0.1,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

In [107]:
simple_instruction_raw_text = """
        Ти - експерт з обробки природної мови. Ваше завдання - обробити задане речення і відповісти на нього реченням зі зміною статі.
        Дотримуйтесь наступних інструкцій, якщо натрапив на слово, яке описує професію(людей) та слова залежні від нього:
                        1. Коли ти зустрічаєш слово чоловічого роду, ти повинен змінити його форму на жіночу, а коли слово належить до жіночого роду, то змінити на чоловічий.
                        2. Якщо це імʼя або прізвище, то їх теж треба змінити на відповідні (можливо випадкові) до форми протилежної статі.
                        3. Ти маєш зберігати відмінки слів, наприклад, якщо слово було в родовому відмінку "режисера", то і слово зі зміненим гендером має бути в родовому відмінку "режисерки".
                        4. Ти маєш зберігати форму множини, якщо слово було в множині ("власники": "власниці") або повертати форму однини, якщо оригінальне слово мало форму однини.
                        5. Фінальні реченння мають мати правильно граматичну структуру та узгодження змінених слів до тих, які від них залежать.
                        6. Виведи змінене речення (без аннотацій) та  пояснення, що ти зробив через "\n"

        Приклади:
        Input: Дмитро Петренко став новим адміністратором за наказом заступниці директора Тетяни Сірої.
        Output: Марина Петренко стала новою адміністраторкою за наказом заступника директорки Володимира Сірого

        Input: Сестри художника Бориса Мартинюка були відомими співачками, тому у Бориса не було часу проводити час з подругами.
        Output: Брати художниці Олени Мартинюк були відомими співаками, тому у Олени не було часу проводити час з подругами.
        Explanation: слово "подругами" вже не стосується сутності JOB, тому змінювати його непотрібно
                        """

In [152]:
simple_instruction_ann_text = """
        Ти - експерт з обробки природної мови. Ваше завдання - обробити задане речення, яке буде надане разом зі словником аннотацій {"слово": "його сутність(label) отримана з NER"} і відповісти на нього реченням зі зміною статі.
        Дотримуйтесь наступних інструкцій, якщо натрапив на слово, яке відноситься до класу "JOB" та слова залежні від нього:
                        1. Коли ти зустрічаєш слово чоловічого роду, ти повинен змінити його форму на жіночу, а коли слово належить до жіночого роду, то змінити на чоловічий.
                        2. Якщо це імʼя або прізвище (відноситься до класу "PERS"), то їх теж треба змінити на більш відповідні до протилежної статі.
                        3. Ти маєш зберігати відмінки слів, наприклад, якщо слово було в родовому відмінку "режисера", то і слово зі зміненим гендером має бути в родовому відмінку "режисерки".
                        4. Ти маєш зберігати форму множини, якщо слово було в множині ("власники": "власниці") або повертати форму однини, якщо оригінальне слово мало форму однини.
                        5. Фінальні реченння мають мати правильно граматичну структуру та узгодження змінених слів до тих, які від них залежать.
                        6. Виведи змінене речення (без аннотацій) та  пояснення, що ти зробив через "\n"

        Приклад:
        Input: Дмитро Петренко став новим адміністратором за наказом заступниці директора Тетяни Сірої. "Дмитро Петренко": PERS, "Тетяни Сірої": PERS, "адміністратором": "JOB", "заступниці": "JOB", "директора": "JOB"
        Output: Марина Петренко стала новою адміністраторкою за наказом заступника директорки Володимира Сірого

        Input: Сестри художника Бориса Мартинюка були відомими співачками, тому у Бориса не було часу проводити час з подругами. "художника": "JOB", "Бориса Мартинюка": "PERS", "співачками": "JOB", "Бориса": "PERS"
        Output: Брати художниці Олени Мартинюк були відомими співаками, тому у Олени не було часу проводити час з подругами.
        Explanation: слово "подругами" вже не стосується сутності JOB, тому змінювати його непотрібно
                        """

In [ ]:
# simple_instruction_ann_text = """
#         Ти - експерт з обробки природної мови. Твоє завдання - обробити задане речення, яке буде надане разом із аннотаціями(отриманими з NER, які будуть вказані в дужках після слова) і відповісти на нього реченням зі зміною статі.
#         Дотримуйтесь наступних інструкцій, якщо натрапив на слово, яке відноситься до класу "JOB" та слова залежні від нього:
#                         1. Коли ти зустрічаєш слово чоловічого роду, ти повинен змінити його форму на жіночу, а коли слово належить до жіночого роду, то змінити на чоловічий.
#                         2. Якщо це імʼя або прізвище (відноситься до класу "PERS"), то їх теж треба змінити на більш відповідні до протилежної статі.
#                         3. Ти маєш зберігати відмінки слів, наприклад, якщо слово було в родовому відмінку "режисера", то і слово зі зміненим гендером має бути в родовому відмінку "режисерки".
#                         4. Ти маєш зберігати форму множини, якщо слово було в множині ("власники": "власниці") або повертати форму однини, якщо оригінальне слово мало форму однини.
#                         5. Фінальні реченння мають мати правильно граматичну структуру та узгодження змінених слів до тих, які від них залежать.
#                         6. Виведи змінене речення(без аннотацій) та пояснення (через "\\n"), чому певні слова були змінені, а які слова змінювати не потрібно було.

#         Приклад:
#         Input: Дмитро Петренко (PERS) став новим адміністратором (JOB) за наказом заступниці(JOB) директора(JOB) Тетяни Сірої (PERS).

#         Input: Сестри художника (JOB) Бориса Мартинюка (PERS) були відомими співачками (JOB), тому у Бориса (PERS) не було часу проводити час з подругами.
#         Output: Брати художниці Олени Мартинюк були відомими співаками, тому у Олени не було часу проводити час з подругами.
#         Explanation: слово "подругами" вже не стосується сутності JOB, тому змінювати його непотрібно
#         """

In [109]:
simple_instruction_raw_text_and_rule_from_pravopys_instruction = """
        Ти - експерт з обробки природної мови. Ваше завдання - обробити задане речення і відповісти на нього реченням зі зміною статі.
        Дотримуйтесь наступних інструкцій, якщо натрапив на слово, яке описує професію(людей) та слова залежні від нього:
                        1. Коли ти зустрічаєш слово чоловічого роду, ти повинен змінити його форму на жіночу, а коли слово належить до жіночого роду, то змінити на чоловічий.
                        2. Якщо це імʼя або прізвище, то їх теж треба змінити на відповідні до форми протилежної статі.
                        3. Ти маєш зберігати відмінки слів, наприклад, якщо слово було в родовому відмінку "режисера", то і слово зі зміненим гендером має бути в родовому відмінку "режисерки".
                        4. Ти маєш зберігати форму множини, якщо слово було в множині ("власники": "власниці") або повертати форму однини, якщо оригінальне слово мало форму однини.
                        5. Фінальні реченння мають мати правильно граматичну структуру та узгодження змінених слів до тих, які від них залежать.
                        6. Виведи змінене речення (без аннотацій) та  пояснення, що ти зробив через "\n"
                        7. Поясни за яким правилом, ти утворив слово для протилежної статі

        Жіночі форми слів можна утворювати за наступними правилами:
                1. За допомогою суфіксів -к-, -иц-(я), -ин-(я), -ес- та ін. від іменників чоловічого роду утворюємо іменники на означення осіб жіночої статі.
                2. Найуживанішим є суфікс -к-, бо він поєднуваний з різними типами основ: авторка, дизайнерка, директорка, редакторка, співачка, студентка, фігуристка та ін.
                3. Суфікс -иц-(я) приєднуємо насамперед до основ на -ник: верстальниця, набі́рниця, порадниця та -ень: учениця. 
                4. Суфікс -ин-(я) сполучаємо з основами на -ець: кравчиня, плавчиня, продавчиня, на приголосний: майстриня, філологиня; бойкиня, лемкиня. Суфікс -ес- рідковживаний: дияконеса, патронеса, поетеса.

        Приклад:
        Input: Дмитро Петренко став новим адміністратором за наказом заступниці директора Тетяни Сірої.
        Output: Марина Петренко стала новою адміністраторкою за наказом заступника директорки Володимира Сірого

        Input: Сестри художника Бориса Мартинюка були відомими співачками, тому у Бориса не було часу проводити час з подругами.
        Output: Брати художниці Олени Мартинюк були відомими співаками, тому у Олени не було часу проводити час з подругами.
        Explanation: слово "подругами" вже не стосується сутності JOB, тому змінювати його непотрібно
                        """

### only raw

In [11]:
model_raw = GenerativeModel(
    "gemini-1.5-flash-001", system_instruction=[simple_instruction_raw_text]
)

In [207]:
for raw_sentence in raw_sentences[:10]:
    res_raw = model_raw.generate_content(
        str(raw_sentence),
        generation_config=generation_config,
        safety_settings=safety_settings,
    )
    print(raw_sentence)
    print(res_raw.text)
    print("-------------------------")

Суд надав слідчим доступ до документації щодо 49 угод, які знаходяться в ТОВ «Армшляхбуд» та управлінні ЖКГ.
Суд надав слідчим доступ до документації щодо 49 угод, які знаходяться в ТОВ «Армшляхбуд» та управлінні ЖКГ.
"Змінено слово "слідчим" на "слідчими", оскільки слово "слідчий" чоловічого роду, а слово "слідчі" жіночого роду. " 

-------------------------
Директором та засновником ТОВ «Армшляхбуд», створеного у лютому 2018 р., є Андранік Мкртчян.
Директоркою та засновницею ТОВ «Армшляхбуд», створеного у лютому 2018 р., є Андранік Мкртчян. 
"Змінено слово "директор" на "директорка", "засновник" на "засновниця", "Андранік" на "Андранік". Збережено відмінки та форму множини." 

-------------------------
Співвласником цієї компанії у свою чергу з часткою в 75% є Іван Плачков, голова наглядової ради ПАТ «Київенерго», яке і проводило тендер.
Співвласницею цієї компанії у свою чергу з часткою в 75% є Івана Плачкова, голова наглядової ради ПАТ «Київенерго», яке і проводило тендер. 
"
Зміне

Try few-shot.

In [222]:
for raw_sentence in raw_sentences[:10]:
    res_raw = model_raw.generate_content(
        str(raw_sentence),
        generation_config=generation_config,
        safety_settings=safety_settings,
    )
    print(raw_sentence)
    print(res_raw.text)
    print("-------------------------")

Суд надав слідчим доступ до документації щодо 49 угод, які знаходяться в ТОВ «Армшляхбуд» та управлінні ЖКГ.
Суд надав слідчим доступ до документації щодо 49 угод, які знаходяться в ТОВ «Армшляхбуд» та управлінні ЖКГ.
Explanation:  зміни не потрібні, бо в реченні немає слів, що описують професії. 

-------------------------
Директором та засновником ТОВ «Армшляхбуд», створеного у лютому 2018 р., є Андранік Мкртчян.
Директоркою та засновницею ТОВ «Армшляхбуд», створеного у лютому 2018 р., є Аніта Мкртчян.
Explanation: змінено слова "директор" на "директорка", "засновник" на "засновниця", "Андранік" на "Аніта" 

-------------------------
Співвласником цієї компанії у свою чергу з часткою в 75% є Іван Плачков, голова наглядової ради ПАТ «Київенерго», яке і проводило тендер.
Співвласницею цієї компанії у свою чергу з часткою в 75% є Олена Плачкова, голова наглядової ради ПАТ «Київенерго», яке і проводило тендер.
Explanation: змінено стать "Іван" на "Олена", "голова" на "голова", "ПАТ" на "

In [118]:
test_sentence = "Дмитро Іванов прописаний за однією адресою в Мелітополі з Оленою Івановою – бізнес-партнера депутата Мелітопольської міської ради від партії «БПП «Солідарність» Сергія Біловола по фірмі «Фундамент-М» (за деякими даними Іванова – сестра Біловола)."

In [228]:
res_raw = model_raw.generate_content(
    str(test_sentence),
    generation_config=generation_config,
    safety_settings=safety_settings,
)
print(test_sentence)
print(res_raw.text)

Дмитро Іванов прописаний за однією адресою в Мелітополі з Оленою Івановою – бізнес-партнера депутата Мелітопольської міської ради від партії «БПП «Солідарність» Сергія Біловола по фірмі «Фундамент-М» (за деякими даними Іванова – сестра Біловола).
Олена Іванова прописана за однією адресою в Мелітополі з Дмитром Івановим – бізнес-партнеркою депутатки Мелітопольської міської ради від партії «БПП «Солідарність» Олени Біловола по фірмі «Фундамент-М» (за деякими даними Іванова – брат Біловола).
Explanation: змінено стать Дмитра на Олену, а Олени на Дмитра, а також змінено стать депутата Сергія на Олену, а також змінено стать "бізнес-партнера" на "бізнес-партнерку" та "сестра" на "брат" 



In [9]:
test_sentence_2 = "Олександр працював математиком в Університеті ім. Тараса Шевченка."

In [13]:
res_raw = model_raw.generate_content(
    str(test_sentence_2),
    generation_config=generation_config,
    safety_settings=safety_settings,
)
print(test_sentence_2)
print(res_raw.text)

Олександр працював математиком в Університеті ім. Тараса Шевченка.
Олександра працювала математичкою в Університеті ім. Тараса Шевченка.
Explanation: змінено слово "математиком" на "математичкою" та ім'я "Олександр" на "Олександра" 



### raw and ann

In [157]:
model_raw_and_ann = GenerativeModel(
    "gemini-1.5-flash-001", system_instruction=[simple_instruction_ann_text]
)

In [146]:
for sentence, ann_dict in test_sentences.items():
    annotated_sentence = annotate_sentence(sentence, ann_dict)
    print(annotated_sentence)
    res_raw = model_raw_and_ann.generate_content(
        str(annotated_sentence),
        generation_config=generation_config,
        safety_settings=safety_settings,
    )
    print(res_raw.text)
    print("--------------------------------")

Найбільшим власником «Спеценергомонтажу (ORG)» є Володимир Ковальов (PERS), брат заступника директора (JOB) дорожньо-будівельної ФПГ «Альтком» (ORG) Михайла Ковальова (PERS).
Найбільшою власницею «Спеценергомонтажу» є Олена Ковальова, сестра заступника директора дорожньо-будівельної ФПГ «Альтком» Михайла Ковальова.
\n
Змінено слова "власником" на "власницею", "Володимир" на "Олена", "брат" на "сестра". Слова "заступника директора" та "Михайла Ковальова" не змінювалися, оскільки вони не стосуються сутності JOB. 

--------------------------------
До 2011 року (DATE) замість Запорожець (PERS) засновником «Спеценергомонтажу (ORG)» був Максим Данилюк (PERS), син депутата (JOB) Донецької облради (ORG) від Партії регіонів (ORG) Івана Данилюка (PERS), який раніше очолював Зуївську ТЕС (ORG), а наразі є радником директора (JOB) «Східенерго (ORG)» з реконструкції та переоснащення.
До 2011 року замість Запорожець засновником «Спеценергомонтажу» була Олена Данилюк, донька депутатки Донецької облра

In [158]:
for sentence, ann_dict in test_sentences.items():
    test_raw_and_annotated = sentence + "\n" + str(ann_dict)
    # annotated_sentence = annotate_sentence(test_raw_and_annotated, ann_dict)
    # print(annotated_sentence)
    print(test_raw_and_annotated)
    res_raw = model_raw_and_ann.generate_content(
        str(test_raw_and_annotated),
        generation_config=generation_config,
        safety_settings=safety_settings,
    )
    print(res_raw.text)
    print("--------------------------------")

Найбільшим власником «Спеценергомонтажу» є Володимир Ковальов, брат заступника директора дорожньо-будівельної ФПГ «Альтком» Михайла Ковальова.
{'Спеценергомонтажу': 'ORG', 'Володимир Ковальов': 'PERS', 'заступника директора': 'JOB', 'ФПГ «Альтком»': 'ORG', 'Михайла Ковальова': 'PERS'}
Найбільшою власницею «Спеценергомонтажу» є Олена Ковальова, сестра заступниці директорки дорожньо-будівельної ФПГ «Альтком» Наталії Ковальової.
Explanation: змінено стать власника та заступника директора, а також їхні імена. 

--------------------------------
До 2011 року замість Запорожець засновником «Спеценергомонтажу» був Максим Данилюк, син депутата Донецької облради від Партії регіонів Івана Данилюка, який раніше очолював Зуївську ТЕС, а наразі є радником директора «Східенерго» з реконструкції та переоснащення.
{'Спеценергомонтажу': 'ORG', '2011 року': 'DATE', 'Запорожець': 'PERS', 'Максим Данилюк': 'PERS', 'депутата': 'JOB', 'Донецької облради': 'ORG', 'Партії регіонів': 'ORG', 'Івана Данилюка': 'P

In [42]:
res_raw = model_raw_and_ann.generate_content(
    str(test_sentence),
    generation_config=generation_config,
    safety_settings=safety_settings,
)
print(test_sentence)
print(res_raw.text)

Дмитро Іванов прописаний за однією адресою в Мелітополі з Оленою Івановою – бізнес-партнера депутата Мелітопольської міської ради від партії «БПП «Солідарність» Сергія Біловола по фірмі «Фундамент-М» (за деякими даними Іванова – сестра Біловола).
Олена Іванова прописана за однією адресою в Мелітополі з Дмитром Івановим – бізнес-партнеркою депутатки Мелітопольської міської ради від партії «БПП «Солідарність» Олени Бівової по фірмі «Фундамент-М» (за деякими даними Іванова – брат Бівової).
Explanation: змінено стать "Дмитро Іванов" на "Олена Іванова", "бізнес-партнера" на "бізнес-партнерку", "депутата" на "депутатку", "Сергія Біловола" на "Олени Бівової", "сестра" на "брат"



In [ ]:
total_tokens_for_text = 0

for i in range(len(raw_sentences)):
    test_raw_and_annotated = str(raw_sentences[i]) + "\n" + str(annotated_sentences[i])

    res_raw_and_annotated = model_raw_and_ann.generate_content(
        test_raw_and_annotated,
        generation_config=generation_config,
        safety_settings=safety_settings,
    )
    print(raw_sentences[i])
    print(res_raw_and_annotated.text)
    print("-------------------------")
    total_tokens_for_text += res_raw_and_annotated.usage_metadata.total_token_count

Суд надав слідчим доступ до документації щодо 49 угод, які знаходяться в ТОВ «Армшляхбуд» та управлінні ЖКГ.
Суд надав слідчим доступ до документації щодо 49 угод, які знаходяться в ТОВ «Армшляхбуд» та управлінні ЖКГ.
"Змінено слово "слідчим" на "слідчими" відповідно до граматичного роду. " 

-------------------------
Директором та засновником ТОВ «Армшляхбуд», створеного у лютому 2018 р., є Андранік Мкртчян.
Директоркою та засновницею ТОВ «Армшляхбуд», створеного у лютому 2018 р., є Андранік Мкртчян. 
"Змінено слово "засновником" на "засновницею", "директором" на "директоркою" та "Андранік Мкртчян" на "Андранік Мкртчян". " 

-------------------------
Співвласником цієї компанії у свою чергу з часткою в 75% є Іван Плачков, голова наглядової ради ПАТ «Київенерго», яке і проводило тендер.
Співвласницею цієї компанії у свою чергу з часткою в 75% є Івана Плачкова, голова наглядової ради ПАТ «Київенерго», яке і проводило тендер. 
"Змінено стать слова "голова" на "голова", а також ім'я "Іван

Try few-shot.

In [224]:
for i in range(10):
    test_raw_and_annotated = str(raw_sentences[i]) + "\n" + str(annotated_sentences[i])

    res_raw_and_annotated = model_raw_and_ann.generate_content(
        test_raw_and_annotated,
        generation_config=generation_config,
        safety_settings=safety_settings,
    )
    print(raw_sentences[i])
    print(res_raw_and_annotated.text)
    print("-------------------------")

Суд надав слідчим доступ до документації щодо 49 угод, які знаходяться в ТОВ «Армшляхбуд» та управлінні ЖКГ.
Суд надав слідчим доступ до документації щодо 49 угод, які знаходяться в ТОВ «Армшляхбуд» та управлінні ЖКГ.
Explanation:  змінено "слідчим" на "слідчими" 

-------------------------
Директором та засновником ТОВ «Армшляхбуд», створеного у лютому 2018 р., є Андранік Мкртчян.
Директоркою та засновницею ТОВ «Армшляхбуд», створеного у лютому 2018 р., є Аніта Мкртчян.
Explanation: змінено стать "директора" на "директорки", "засновника" на "засновниці", "Андранік Мкртчян" на "Аніта Мкртчян"

-------------------------
Співвласником цієї компанії у свою чергу з часткою в 75% є Іван Плачков, голова наглядової ради ПАТ «Київенерго», яке і проводило тендер.
Співвласницею цієї компанії у свою чергу з часткою в 75% є Олена Плачкова, голова наглядової ради ПАТ «Київенерго», яке і проводило тендер.
Explanation: змінено стать "Іван Плачков" на "Олена Плачкова", "голова наглядової ради" на "гол

### raw and rules

In [22]:
model_raw_and_rules = GenerativeModel(
    "gemini-1.5-flash-001",
    system_instruction=[simple_instruction_raw_text_and_rule_from_pravopys_instruction],
)

In [226]:
for raw_sentence in raw_sentences[:10]:
    res_raw_and_rules = model_raw_and_rules.generate_content(
        str(raw_sentence),
        generation_config=generation_config,
        safety_settings=safety_settings,
    )
    print(raw_sentence)
    print(res_raw_and_rules.text)
    print("-------------------------")

Суд надав слідчим доступ до документації щодо 49 угод, які знаходяться в ТОВ «Армшляхбуд» та управлінні ЖКГ.
Суд надав слідчим доступ до документації щодо 49 угод, які знаходяться в ТОВ «Армшляхбуд» та управлінні ЖКГ.
Explanation:  зміни не потрібні, бо в реченні немає слів, що описують професії. 

-------------------------
Директором та засновником ТОВ «Армшляхбуд», створеного у лютому 2018 р., є Андранік Мкртчян.
Директоркою та засновницею ТОВ «Армшляхбуд», створеного у лютому 2018 р., є Аніта Мкртчян.
Explanation: змінено слова "директор" на "директорка", "засновник" на "засновниця", "Андранік" на "Аніта" 

-------------------------
Співвласником цієї компанії у свою чергу з часткою в 75% є Іван Плачков, голова наглядової ради ПАТ «Київенерго», яке і проводило тендер.
Співвласницею цієї компанії у свою чергу з часткою в 75% є Олена Плачкова, голова наглядової ради ПАТ «Київенерго», яке і проводило тендер.
Explanation: змінено стать "Іван Плачков" на "Олена Плачкова", "голова" на "го

In [ ]:
res_raw_and_rules = model_raw_and_rules.generate_content(
    str(test_sentence_2),
    generation_config=generation_config,
    safety_settings=safety_settings,
)

print(test_sentence_2)
print(res_raw_and_rules.text)

Олександр працював математиком в Університеті ім. Тараса Шевченка.
Олександра працювала математичкою в Університеті ім. Тараса Шевченка.
Explanation: слово "математиком" змінено на "математичкою" за правилом 1, 2, 3, 4, 5, 6, 7.



In [23]:
res_raw_and_rules = model_raw_and_rules.generate_content(
    "математик", generation_config=generation_config, safety_settings=safety_settings
)

print(res_raw_and_rules.text)

математичка
Explanation: слово "математик" чоловічого роду, тому його змінили на жіночий рід за допомогою суфікса -к-



In [24]:
res_raw_and_rules = model_raw_and_rules.generate_content(
    "суддя", generation_config=generation_config, safety_settings=safety_settings
)

print(res_raw_and_rules.text)

суддя
Explanation:  слово "суддя" вже є жіночого роду, тому його не потрібно змінювати. 



In [26]:
res_raw_and_rules = model_raw_and_rules.generate_content(
    "оздоблювальник",
    generation_config=generation_config,
    safety_settings=safety_settings,
)

print(res_raw_and_rules.text)

оздоблювальниця
Explanation: слово "оздоблювальник" - це слово чоловічого роду, тому для утворення жіночого роду до нього додаємо суфікс -иц-



In [27]:
res_raw_and_rules = model_raw_and_rules.generate_content(
    "травник", generation_config=generation_config, safety_settings=safety_settings
)

print(res_raw_and_rules.text)

травниця
Explanation: слово "травник" - це чоловічий рід, тому для жіночого роду додаємо суфікс -иц-



In [41]:
with open("../list.txt", "r") as file:
    # Read all lines and store them in a list
    lines = file.read().splitlines()

In [42]:
lines

['заступника директора',
 'депутата',
 'радником директора',
 'екс-нардеп',
 'екс-власник',
 'екс-міністрів',
 'народного депутата',
 'співзасновником',
 'замкерівника',
 'голови правління',
 'екс-депутата',
 'начальником виробничо-технічного відділу',
 'очільник',
 'Екс-керівник',
 'голови',
 'екс-очільника',
 'чиновника',
 'засновників',
 'мера',
 'депутатом',
 'депутатом',
 'директором',
 'директором',
 'оператор',
 'страховиків',
 'атомників',
 'Комунальники',
 'співзасновником',
 'депутат',
 'директора',
 "екс-прем'єр-міністра",
 'керівництво',
 'Податківці',
 'заступник начальника',
 'екс-податківець',
 'першим заступником начальника',
 'радником голови',
 'голови митниці',
 'екс-віце-спікера',
 'екс-заступник голови',
 'журналістам',
 'депутата',
 'голови',
 'Засновниками',
 'засновниками',
 'голову',
 'Слідчий',
 'голова',
 'мер',
 'слідчий',
 'Міським головою',
 'засновником',
 'мер',
 'начальник',
 'фахівець з набору та отримання землі',
 'екс-нардеп',
 'заступником начальник

In [40]:
res_raw_and_rules = model_raw.generate_content(
    "отаман", generation_config=generation_config, safety_settings=safety_settings
)

print(res_raw_and_rules.text)

отаманка 
Explanation: змінено слово "отаман" на "отаманка" 



In [39]:
res_raw_and_rules = model_raw_and_rules.generate_content(
    "отаман", generation_config=generation_config, safety_settings=safety_settings
)

print(res_raw_and_rules.text)

отаманка
Explanation: слово "отаман" - чоловічого роду, тому для жіночого роду використовуємо суфікс -к-



In [45]:
for word in lines:
    print(word)
    res_raw_and_rules = model_raw_and_rules.generate_content(
        word, generation_config=generation_config, safety_settings=safety_settings
    )
    print("raw:", res_raw_and_rules.text)

    res_raw_and_rules = model_raw.generate_content(
        word, generation_config=generation_config, safety_settings=safety_settings
    )

    print("rules:", res_raw_and_rules.text)

    print("--------------------------------------")

заступника директора
raw: заступниці директорки
Explanation: змінив слово "заступника" на "заступниці", а слово "директора" на "директорки"
За правилом 1, 2, 3, 4, 5, 6, 7:  змінив слово "заступника" на "заступниці" за правилом 1, 2, 3, 4, 5, 6, 7, а слово "директора" на "директорки" за правилом 1, 2, 3, 4, 5, 6, 7. 

rules: заступниці директорки
Explanation: змінено слово "заступника" на "заступниці", а слово "директора" на "директорки" 

--------------------------------------
депутата
raw: депутатка
Explanation: слово "депутата" змінено на "депутатка" за правилом 1, додавши суфікс -к-

rules: депутатку
Explanation: змінено слово "депутата" на "депутатку" відповідно до жіночого роду. 

--------------------------------------
радником директора
raw: радницею директорки
Explanation: слово "директор" змінено на "директорка" за правилом 1, слово "радником" змінено на "радницею" за правилом 1, суфікс -к-

rules: радницею директорки
Explanation: змінено слово "радником" на "радницею" та "дир

KeyboardInterrupt: 

## GPT 4o-mini

In [31]:
import openai
from openai import OpenAI

import os
from dotenv import load_dotenv

load_dotenv(".env")

openai.api_key = os.environ.get("OPENAI_API_KEY")

In [ ]:
# Виведіть змінене речення (без анотацій) і коротке пояснення через "\n" щодо внесених змін.
# Explanation: слово "подругами" вже не стосується сутності JOB, тому змінювати його непотрібно

In [50]:
updated_prompt = """
Ви - експерт з обробки природної мови. Ваше завдання - змінити задане речення зі зміною статі, використовуючи словник анотацій у форматі {"слово": "його сутність(label) з NER"}. Дотримуйтесь таких правил:
    1. Якщо зустрічається слово чоловічого роду з класу "JOB", змініть його на жіночий аналог, і навпаки для жіночого роду.
    2. Імена та прізвища (клас "PERS") також слід змінювати на випадкові протилежної статі.
    3. Зберігайте відмінки слів: якщо слово було, наприклад, у родовому відмінку ("режисера"), то його аналог має теж бути в родовому відмінку ("режисерки").
    4. Зберігайте форму слова за числом: множина залишається множиною ("власники" → "власниці"), однина - одниною.
    5. Забезпечте граматичну коректність речення, узгоджуючи змінені слова з контекстом.
    6. Як результат виведи змінене речення без жодних додаткових пояснень.

    Приклад:
        Input: Дмитро Петренко став новим адміністратором за наказом заступниці директора Тетяни Сірої. {"Дмитро Петренко": PERS, "Тетяни Сірої": PERS, "адміністратором": "JOB", "заступниці": "JOB", "директора": "JOB"}
        Output: Марина Петренко стала новою адміністраторкою за наказом заступника директорки Володимира Сірого.

        Input: Сестри художника Бориса Мартинюка були відомими співачками, тому у Бориса не було часу проводити час з подругами. {"художника": "JOB", "Бориса Мартинюка": "PERS", "співачками": "JOB", "Бориса": "PERS"}
        Output: Брати художниці Олени Мартинюк були відомими співаками, тому у Олени не було часу проводити час з подругами.
        
"""

In [ ]:
sys_prompt = """
        Ти - експерт з обробки природної мови. Ваше завдання - обробити задане речення, яке буде надане разом зі словником аннотацій {"слово": "його сутність(label) отримана з NER"} і відповісти на нього реченням зі зміною статі.
        Дотримуйтесь наступних інструкцій, якщо натрапив на слово, яке відноситься до класу "JOB" та слова залежні від нього:
                        1. Коли ти зустрічаєш слово чоловічого роду, ти повинен змінити його форму на жіночу, а коли слово належить до жіночого роду, то змінити на чоловічий.
                        2. Якщо це імʼя або прізвище (відноситься до класу "PERS"), то їх теж треба змінити на більш відповідні до протилежної статі.
                        3. Ти маєш зберігати відмінки слів, наприклад, якщо слово було в родовому відмінку "режисера", то і слово зі зміненим гендером має бути в родовому відмінку "режисерки".
                        4. Ти маєш зберігати форму множини, якщо слово було в множині ("власники": "власниці") або повертати форму однини, якщо оригінальне слово мало форму однини.
                        5. Фінальні реченння мають мати правильно граматичну структуру та узгодження змінених слів до тих, які від них залежать.
                        6. Виведи змінене речення (без аннотацій) та  пояснення, що ти зробив через "\n"

        Приклад:
        Input: Дмитро Петренко став новим адміністратором за наказом заступниці директора Тетяни Сірої. "Дмитро Петренко": PERS, "Тетяни Сірої": PERS, "адміністратором": "JOB", "заступниці": "JOB", "директора": "JOB"
        Output: Марина Петренко стала новою адміністраторкою за наказом заступника директорки Володимира Сірого

        Input: Сестри художника Бориса Мартинюка були відомими співачками, тому у Бориса не було часу проводити час з подругами. "художника": "JOB", "Бориса Мартинюка": "PERS", "співачками": "JOB", "Бориса": "PERS"
        Output: Брати художниці Олени Мартинюк були відомими співаками, тому у Олени не було часу проводити час з подругами.
        Explanation: слово "подругами" вже не стосується сутності JOB, тому змінювати його непотрібно
                        """

In [78]:
# messages = [
#     {
#         "role": "system",
#         "content": updated_prompt
#         # "content": """
#         # Ти - експерт з обробки природної мови. Твоє завдання - обробити задане речення, яке буде надане разом із аннотаціями(отриманими з NER, які будуть вказані в дужках після слова) і відповісти на нього реченням зі зміною статі.
#         # Дотримуйтесь наступних інструкцій, якщо натрапив на слово, яке відноситься до класу "JOB" та слова залежні від нього:
#         #                 1. Коли ти зустрічаєш слово чоловічого роду, ти повинен змінити його форму на жіночу, а коли слово належить до жіночого роду, то змінити на чоловічий.
#         #                 2. Якщо це імʼя або прізвище (відноситься до класу "PERS"), то їх теж треба змінити на більш відповідні до протилежної статі.
#         #                 3. Ти маєш зберігати відмінки слів, наприклад, якщо слово було в родовому відмінку "режисера", то і слово зі зміненим гендером має бути в родовому відмінку "режисерки".
#         #                 4. Ти маєш зберігати форму множини, якщо слово було в множині ("власники": "власниці") або повертати форму однини, якщо оригінальне слово мало форму однини.
#         #                 5. Фінальні реченння мають мати правильно граматичну структуру та узгодження змінених слів до тих, які від них залежать.
#         #                 6. Виведи змінене речення(без аннотацій) та пояснення (через "\\n"), чому слова, які стосуються скласів JOB і PERS були змінені або не змінені.

#         # Приклад:
#         # Input: Дмитро Петренко (PERS) став новим адміністратором (JOB) за наказом заступниці(JOB) директора(JOB) Тетяни Сірої (PERS).

#         # Input: Сестри художника (JOB) Бориса Мартинюка (PERS) були відомими співачками (JOB), тому у Бориса (PERS) не було часу проводити час з подругами.
#         # Output: Брати художниці Олени Мартинюк були відомими співаками, тому у Олени не було часу проводити час з подругами.
#         # Explanation: слово "подругами" вже не стосується сутності JOB, тому змінювати його непотрібно
#         # """
#     }
# ]


def change_gender(sentence):
    # messages.append(
    #     {
    #         "role": "user",
    #         "content": sentence
    #     }
    # )

    messages = [
        {"role": "system", "content": updated_prompt},
        {"role": "user", "content": sentence},
    ]

    response = openai.chat.completions.create(
        model="gpt-4o-mini", messages=messages, temperature=0.8, max_tokens=200, top_p=1
    )

    response_content = response.choices[0].message.content
    return response_content

In [ ]:
# def annotate_sentence(sentence, ng_sentence_with_jobs):
#     sorted_entities = sorted(ng_sentence_with_jobs.keys(), key=len, reverse=True)

#     for entity in sorted_entities:
#         label = ng_sentence_with_jobs[entity]
#         annotated_entity = f"{entity} ({label})"
#         sentence = sentence.replace(entity, annotated_entity)

#     return sentence

In [ ]:
# test_sentence = dict(list(ng_sentence_with_jobs.items())[3:4])

In [141]:
test_sentence.keys()

dict_keys(["На придбання акцій «Донбасенерго» Ігор Гуменюк позичив 732 млн. під 3.5% річних у структури, яка використовувала телефони і e-mail фірми «Голден Деррік», що пов'язана із бізнес-середовищем екс-міністрів Миколи Присяжнюка, Едуарда Ставицького і народного депутата Іванющенка."])

In [ ]:
# for sentence, ann_dict in test_sentence.items():
#     annotated_sentence = annotate_sentence(sentence, ann_dict)
#     print(annotated_sentence)
#     output = change_gender(annotated_sentence, messages)
#     print(output)

На придбання акцій «Донбасенерго (ORG)» Ігор Гуменюк (PERS) позичив 732 млн. (MON) під 3.5% (PCT) річних у структури, яка використовувала телефони і e-mail фірми «Голден Деррік (ORG)», що пов'язана із бізнес-середовищем екс-міністрів (JOB) Миколи Присяжнюка (PERS), Едуарда Ставицького (PERS) і народного депутата (JOB) (JOB) Іванющенка (PERS).
ChatCompletion(id='chatcmpl-AQLSbw1LBpjDoRPNZE8Bt5uYzqb99', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='На придбання акцій «Донбасенерго» Ірина Гуменюк позичила 732 млн. під 3.5% річних у структури, яка використовувала телефони і e-mail фірми «Голден Деррік», що пов\'язана із бізнес-середовищем екс-міністрів Олени Присяжнюк, Едуарда Ставицького і народного депутата (JOB) Іванющенка.\n\nExplanation: \n- "Ігор" змінено на "Ірина" для зміни статі з чоловічої на жіночу (PERS).\n- "Гуменюк" залишено без змін, оскільки це прізвище, яке зазвичай не має гендерної прив\'язки.\n- "Микола" змінено на "

In [ ]:
# for sentence, ann_dict in test_sentence.items():
#     annotated_sentence = annotate_sentence(sentence, ann_dict)
#     print(annotated_sentence)
#     output = change_gender(annotated_sentence, messages)
#     print(output)

На придбання акцій «Донбасенерго (ORG)» Ігор Гуменюк (PERS) позичив 732 млн. (MON) під 3.5% (PCT) річних у структури, яка використовувала телефони і e-mail фірми «Голден Деррік (ORG)», що пов'язана із бізнес-середовищем екс-міністрів (JOB) Миколи Присяжнюка (PERS), Едуарда Ставицького (PERS) і народного депутата (JOB) (JOB) Іванющенка (PERS).
На придбання акцій «Донбасенерго» Ірина Гуменюк позичила 732 млн. під 3.5% річних у структури, яка використовувала телефони і e-mail фірми «Голден Деррік», що пов'язана із бізнес-середовищем екс-міністерок Миколи Присяжнюка, Едуарда Ставицького і народного депутата Іванющенка.

Explanation: 
- "Ігор" змінено на "Ірина", оскільки це ім'я чоловічого роду, було змінено на жіноче.
- "Гуменюк" залишено незмінним, оскільки прізвище не має гендерної специфіки, але в контексті ім'я стало жіночим.
- "екс-міністрів" змінено на "екс-міністер


In [143]:
test_sentences = dict(list(ng_sentence_with_jobs.items())[:20])

In [ ]:
# for sentence, ann_dict in test_sentences.items():
#     annotated_sentence = annotate_sentence(sentence, ann_dict)
#     print(annotated_sentence)
#     output = change_gender(annotated_sentence, messages)
#     print(output)
#     print("-------------------------------------")

Найбільшим власником «Спеценергомонтажу (ORG)» є Володимир Ковальов (PERS), брат заступника директора (JOB) дорожньо-будівельної ФПГ «Альтком» (ORG) Михайла Ковальова (PERS).
Найбільшою власницею «Спеценергомонтажу» є Володимира Ковальова, сестра заступниці директорки дорожньо-будівельної ФПГ «Альтком» Михайла Ковальової.

Explanation:
- "Володимир" змінено на "Володимира", оскільки це ім'я чоловічого роду, було змінено на жіноче.
- "Ковальов" залишено незмінним, оскільки прізвище не має гендерної специфіки, але в контексті ім'я стало жіночим.
- "заступника директора" змінено на "заступниці директорки", оскільки "заступник" чоловічого роду, було змінено на жіночий.
- "брат" змінено на "
-------------------------------------
До 2011 року (DATE) замість Запорожець (PERS) засновником «Спеценергомонтажу (ORG)» був Максим Данилюк (PERS), син депутата (JOB) Донецької облради (ORG) від Партії регіонів (ORG) Івана Данилюка (PERS), який раніше очолював Зуївську ТЕС (ORG), а наразі є радником ди

In [162]:
for sentence, ann_dict in test_sentences.items():
    test_raw_and_annotated = sentence + "\n" + str(ann_dict)
    # print(test_raw_and_annotated)
    # annotated_sentence = annotate_sentence(test_raw_and_annotated, ann_dict)
    print(test_raw_and_annotated)
    output = change_gender(test_raw_and_annotated, messages)
    print(output)
    print("-------------------------------------")

Найбільшим власником «Спеценергомонтажу» є Володимир Ковальов, брат заступника директора дорожньо-будівельної ФПГ «Альтком» Михайла Ковальова.
{'Спеценергомонтажу': 'ORG', 'Володимир Ковальов': 'PERS', 'заступника директора': 'JOB', 'ФПГ «Альтком»': 'ORG', 'Михайла Ковальова': 'PERS'}
Найбільшою власницею «Спеценергомонтажу» є Валентина Ковальова, сестра заступниці директорки дорожньо-будівельної ФПГ «Альтком» Марії Ковальової. 

Explanation: 
- "Володимир Ковальов" змінено на "Валентина Ковальова" (ім'я чоловічого роду на жіноче).
- "заступника директора" змінено на "заступниці директорки" (чоловічий рід на жіночий).
- "Михайла Ковальова" змінено на "Марії Ковальової" (ім'я чоловічого роду на жіноче). 
- Слова, які не відносяться до JOB чи PERS, залишилися без змін.
-------------------------------------
До 2011 року замість Запорожець засновником «Спеценергомонтажу» був Максим Данилюк, син депутата Донецької облради від Партії регіонів Івана Данилюка, який раніше очолював Зуївську ТЕС

## Nazar's codes

In [ ]:
# ignore logs
# Nazar's code
logging.getLogger("stanza").setLevel(logging.ERROR)


def changer(morph_token, gender, token):
    # комунікабельний -> комунікабельна
    if morph_token.tag.POS == "VERB" or morph_token.tag.POS == "ADJF":
        if morph_token.tag.gender == "masc" and gender == "femn":
            token = morph_token.inflect({"femn"}).word

        elif morph_token.tag.gender == "femn" and gender == "masc":
            token = morph_token.inflect({"masc"}).word

        else:
            token = morph_token.word
    return token


def gender_changer(text: str, gender: str = "masc") -> str:
    """
    Function for changing verbs from masculine to feminine and vice versa for Ukrainian language

    Args:
        text: Ukrainian language text
        gender: gender to which the verbs should be changed. Default: male

    Returns:
        text with changed verbs to masculine or feminine
    """

    nlp = stanza.Pipeline(lang="uk", processors="tokenize,mwt,pos,lemma,depparse")
    text_prep = text.replace(":", ".").replace(";", ".")
    doc = nlp(text_prep)
    morph = pymorphy3.MorphAnalyzer(lang="uk")
    changer_count = 0
    for sentence in doc.sentences:
        # find head for я with deprel to nsubj
        has_nsubj = any(
            [
                word.deprel == "nsubj" and word.upos not in ["ADJ", "NOUN"]
                for word in sentence.words
            ]
        )
        head_ids = []
        for word in sentence.words:
            if word.text.lower() == "я" and word.deprel == "nsubj":
                head_ids.append(word.head)
            # print(word.text, word.deprel, word.upos, word.head)

        if not has_nsubj or head_ids != []:
            for word in sentence.words:
                token = word.text
                if token.lower() == "є":
                    break
                morph_token = morph.parse(token)[0]
                if (
                    (word.deprel == "root" and sentence.words[word.head].upos != "NOUN")
                    or (word.id in head_ids)
                    or (
                        (word.upos == "ADJ") and (word.deprel != "amod" or word.id != 1)
                    )
                ):
                    token = changer(morph_token, gender, token)
                    if word.id == 1:
                        token = token.capitalize()
                    text = (
                        text[: word.start_char + changer_count]
                        + token
                        + text[word.end_char + changer_count :]
                    )
                    changer_count += len(token) - len(word.text)
    return text

# NER (DEV and TEST)

In [7]:
def read_data(file_path):
    data_dict = {}
    current_label = None

    with open(file_path, "r") as file:
        for line in file:
            line = line.strip()

            if line in ["DEV", "TEST"]:
                current_label = line
                data_dict[current_label] = []
            elif current_label:
                if line:
                    data_dict[current_label].append(line)

    return data_dict

In [8]:
file_path = "/Users/linndfors/study/diploma/ner_for_fem/data/dev-test-split.txt"
data_dict = read_data(file_path)
print(data_dict)

{'DEV': ['e09cf46f8336', 'c0c436cc9cb6', 'bdda872d3ef9', 'e010e86e436c', '1448663b16b2', '6b5d4229b548', '91024fa21059', '10a2da5c514c', 'b6756b05bab3', 'c3402705f448', '350d55a9cd57', 'facfaebdda53', 'd07dfb774c28', '0b962bee49bc', '2f18f73184c6', 'bd86f567c746', 'dc1202fd9850', 'e9061171da9d', '2846bf1d66a8', '9c0774172f73', '38603af07530', 'd81f996d0ab8', '15238b87db03', '74a7e2a340dd', '232be2c6b5c2', 'db336c963a75', 'cff8a0408e5d', '9fbfe077a27a', '3ba7ca929d68', 'c28168a23912', 'dc25535e13d7', 'c6807dbff658', '1001d3d44d5d', '561acef3f531', '27a1722d517c', '5085bb95b51a', '5a0aedfe0f5d', '54463e60c449', '8cc49a8b2d76', '209e9f10fc09', '38a3cffd4beb', 'c069c0fe5b09', '4067c1915516', 'c375246adb0a', '68f6d1cfc486', '53332fd5b311', '766e1942f398', '9e763fde7f9d', 'b09a79bafe6c', '095a5debd3eb', '9047cfab803a', 'abb07434ad17', 'b576ac602665', 'ff069c11e13d', 'dac0de834f47', 'e053b1d934d3', '49752e89b7ff', 'a487d1895726', 'f4136bfda19a', 'd530671576f8', '79812f4abec1', '164940c99b86',

In [9]:
def find_files(directory):
    dev_list = []
    test_list = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".txt"):
                if file.split(".")[0] in list(data_dict["DEV"]):
                    file_path = os.path.join(root, file)
                    dev_list.append(file_path)
                elif file.split(".")[0] in list(data_dict["TEST"]):
                    file_path = os.path.join(root, file)
                    test_list.append(file_path)

    return dev_list, test_list


def read_files(txt_files):
    text = ""
    for txt_file in txt_files:
        with open(txt_file, "r", encoding="utf-8") as txt_f:
            text += txt_f.read()

    return text

In [10]:
bruk_dev, bruk_test = find_files("/Users/linndfors/study/diploma/ner_for_fem/data/bruk")
ng_dev, ng_test = find_files("/Users/linndfors/study/diploma/ner_for_fem/data/ng")

dev_files = bruk_dev + ng_dev
test_files = bruk_test + ng_test

In [11]:
dev_files_text = read_files(dev_files)
test_files_text = read_files(test_files)

dev_ng_text = read_files(ng_dev)
test_ng_text = read_files(ng_test)

dev_bruk_text = read_files(bruk_dev)
test_bruk_text = read_files(bruk_test)

In [12]:
test_bruk_text

"Перш ніж готувати з м’яса , треба навчитися його вибирати .\n\nА вибирати слід молоде , а не старе .\n\nАби відрізнити молоде м’ясо від старого , можна подивитися на товщину ребер : якщо ребро тонке , то м’ясо молоде , якщо ж ребро широке ( розміром із сірникове пуделко ) — то м’ясо старе .\n\nТакож м’ясо потрібно оглянути і помацати — у несвіжому м’ясі при натисканні залишиться вм’ятина .\n\nМ’ясо , яке заморожували і розморожували кілька разів , буде вологим .\n\nЯкщо м’ясо нашприцьоване поліфосфатами , то виглядатиме воно так , ніби здулося .\n\nЗаморожене м’ясо дешевше від охолодженого .\n\nРозморожувати м’ясо краще в холодильнику при температурі 0-2 градуси .\n\nПри різкому заморожуванні у м’ясі зберігаються і вітаміни , і мікроелементи , і ферменти , і всі корисні біологічні властивості .\n\nТому при розморожуванні маємо цілком добрий продукт , не гірший за свіже м’ясо .\n\nА от кілька разів заморожене м’ясо практично втрачає свої властивості і його споживання може бути небезпеч

In [13]:
def get_sentences_with_job_titles(txt_file_names):

    sentence_with_jobs = {}

    for txt_file in txt_file_names:
        ann_file = txt_file.replace(".txt", ".ann")
        labeled_sentences_list, sentences = extract_entities(txt_file, ann_file)
        for i in range(len(sentences)):
            sentence_with_jobs[sentences[i]] = labeled_sentences_list[i]

    return sentence_with_jobs

In [ ]:
ng_dev_sentence_with_jobs = get_sentences_with_job_titles(ng_dev)
ng_test_sentence_with_jobs = get_sentences_with_job_titles(ng_test)

In [74]:
# for sent, ann_dict in ng_dev_sentence_with_jobs.items():
#     changed_sent = sent
#     for ent, ann in ann_dict.items():
#         changed_ent = f'({ent}: {ann})'
#         changed_sent = changed_sent.replace(ent, changed_ent)
#     print(changed_sent)

In [58]:
len(ng_dev_sentence_with_jobs)

726

In [77]:
len(ng_test_sentence_with_jobs)

303

In [ ]:
# TODO feed only PERS and JOB annotations (TO COMPARE)

In [ ]:
gender_swaped_pairs = {}

for sentence, ann_dict in tqdm(
    ng_dev_sentence_with_jobs.items(), desc="Processing sentences"
):
    try:
        test_raw_and_annotated = sentence + "\n" + str(ann_dict)
        # print(test_raw_and_annotated)
        output = change_gender(test_raw_and_annotated)
        # print(output)
        # print("-------------------------------------")
        gender_swaped_pairs[sentence] = output
    except Exception as e:
        print("error:", e)

In [83]:
len(gender_swaped_pairs)

712

In [84]:
gender_swaped_df = pd.DataFrame(gender_swaped_pairs.items())

In [85]:
gender_swaped_df.head()

,0,1
0,"Слідчі встановили, що у 2018-2019 рр. управлін...","Слідчі встановили, що у 2018-2019 рр. управлін..."
1,Суд надав слідчим доступ до документації щодо ...,Суд надав слідчим доступ до документації щодо ...
2,"Директором та засновником ТОВ «Армшляхбуд», ст...","Директоркою та засновницею ТОВ «Армшляхбуд», с..."
3,"Жовтневий райсуд м. Дніпро призначив 25,5 тис ...","Жовтневий райсуд м. Дніпро призначив 25,5 тис ..."
4,В подальшому Тютюнник пообіцяв директору ПТ «А...,В подальшому Тютюнник пообіцяла директорці ПТ ...


In [ ]:
# gender_swaped_df.to_csv("ng_test_swapped.csv")

In [79]:
gender_swaped_pairs_test = {}

for sentence, ann_dict in tqdm(
    ng_test_sentence_with_jobs.items(), desc="Processing sentences"
):
    try:
        test_raw_and_annotated = sentence + "\n" + str(ann_dict)
        # print(test_raw_and_annotated)
        output = change_gender(test_raw_and_annotated)
        print(output)
        print("-------------------------------------")
        gender_swaped_pairs_test[sentence] = output
    except Exception as e:
        print("error:", e)

Processing sentences:   0%|          | 1/303 [00:01<09:08,  1.82s/it]

Співвласницею цієї компанії у свою чергу з часткою в 75% є Ірина Плачкова, голова наглядової ради ПАТ «Київенерго», яке і проводило тендер.
-------------------------------------


Processing sentences:   1%|          | 2/303 [00:03<08:21,  1.67s/it]

Одеситка Ірина Плачкова почала працювати в компанії понад 20 років тому, з середини 90-х займає у ній керівні посади, у 1995-99 вперше стає головою правління «Київенерго».
-------------------------------------


Processing sentences:   1%|          | 3/303 [00:05<08:44,  1.75s/it]

...

Директорка «України» Ростислава Левінзон, призначена IBRC ще в листопаді 2011 року, змогла приступити до обов'язків тільки в цьому році.
-------------------------------------


Processing sentences: 100%|█████████▉| 302/303 [07:12<00:01,  1.83s/it]

Але через шість років – в жовтні 2012-го Апеляційний суд Кіровоградської області вирішив, що договір між військовими і «Агрорембудсервіс» є нікчемною.
-------------------------------------


Processing sentences: 100%|██████████| 303/303 [07:15<00:00,  1.44s/it]

З 2005 по 2012 роки Юлія Протівцева була директоркою філії ДП Міноборони «Укрвійськбуд» «1080 Управління начальниці робіт», з котрим наразі й судиться за майно.
-------------------------------------


In [ ]:
import csv

In [ ]:
# вичитую змінені пари речень

orig_and_changed_dict = {}

with open('ng_test_swapped.csv', mode='r', encoding='utf-8') as file:
    reader = csv.reader(file)
    
    next(reader)
    for row in reader:
        key = row[1]
        value = row[2]
        orig_and_changed_dict[key] = value

print(orig_and_changed_dict)

{'Співвласником цієї компанії у свою чергу з часткою в 75% є Іван Плачков, голова наглядової ради ПАТ «Київенерго», яке і проводило тендер.': 'Співвласницею цієї компанії у свою чергу з часткою в 75% є Ірина Плачкова, голова наглядової ради ПАТ «Київенерго», яке і проводило тендер.', 'Одесит Іван Плачков почав працювати в компанії понад 20 років тому, з середини 90-х займає у ній керівні посади, у 1995-99 вперше стає головою правління «Київенерго».': 'Одеситка Ірина Плачкова почала працювати в компанії понад 20 років тому, з середини 90-х займає у ній керівні посади, у 1995-99 вперше стає головою правління «Київенерго».', 'У 1999 очолив Міністерство енергетики України, потім – знову «Київенерго» у 2000-2005 р. р. ; у 2005-2006 міністр палива та енергетики України ; багаторазовий депутат Київської міськради, з 2008 – у фракції Кличка.': 'У 1999 очолила Міністерство енергетики України, потім – знову «Київенерго» у 2000-2005 р. р. ; у 2005-2006 міністерка палива та енергетики України ; ба

In [15]:
ng_test_all_sent = test_ng_text.split("\n")

In [16]:
# change original text with the swapped sentences

new_text = []

for x in ng_test_all_sent:
    if x in orig_and_changed_dict.keys():
        new_text.append(orig_and_changed_dict[x])
        print("here:", x, " ----to----", orig_and_changed_dict[x])
    else:
        new_text.append(x)

here: Співвласником цієї компанії у свою чергу з часткою в 75% є Іван Плачков, голова наглядової ради ПАТ «Київенерго», яке і проводило тендер.  ----to---- Співвласницею цієї компанії у свою чергу з часткою в 75% є Ірина Плачкова, голова наглядової ради ПАТ «Київенерго», яке і проводило тендер.
here: Одесит Іван Плачков почав працювати в компанії понад 20 років тому, з середини 90-х займає у ній керівні посади, у 1995-99 вперше стає головою правління «Київенерго».  ----to---- Одеситка Ірина Плачкова почала працювати в компанії понад 20 років тому, з середини 90-х займає у ній керівні посади, у 1995-99 вперше стає головою правління «Київенерго».
here: У 1999 очолив Міністерство енергетики України, потім – знову «Київенерго» у 2000-2005 р. р. ; у 2005-2006 міністр палива та енергетики України ; багаторазовий депутат Київської міськради, з 2008 – у фракції Кличка.  ----to---- У 1999 очолила Міністерство енергетики України, потім – знову «Київенерго» у 2000-2005 р. р. ; у 2005-2006 міністе

In [17]:
"\n".join(new_text)

"ПАТ «Київенерго» 8 травня за результатами тендеру уклало угоду з ДП «Енергоремонт» на ремонт і техобслуговування машин загального призначення вартістю 18,15 млн грн.\nПро це повідомляється у «Віснику державних закупівель».\nУпродовж року фірма має надавати послуги з ремонту обладнання теплофікаційних енергоблоків №1,2 і ТФУ в складі енергоблоків, а також загальностанційного обладнання ТЕЦ-6 СВП «Київські ТЕЦ».\nОплачувати послуги будуть з коштів самого ПАТ «Київенерго», яке належить групі Ріната Ахметова.\n«Енергоремонт» є дочірньою фірмою ПАТ «Київенергоремонт», яка на 57% належить ТОВ «Ресет».\nСпіввласницею цієї компанії у свою чергу з часткою в 75% є Ірина Плачкова, голова наглядової ради ПАТ «Київенерго», яке і проводило тендер.\nОдеситка Ірина Плачкова почала працювати в компанії понад 20 років тому, з середини 90-х займає у ній керівні посади, у 1995-99 вперше стає головою правління «Київенерго».\nУ 1999 очолила Міністерство енергетики України, потім – знову «Київенерго» у 2000

In [18]:
import shutil

In [122]:
def change_orig_text_to_gend_swapped(txt_file):
    # create a new files with the old name but new text with swapped sentences. also add old ann file
    file_and_text_dict = {}

    ann_file = txt_file.replace(".txt", ".ann")
    changed_ann_file = ann_file.replace('ng', "ng_changed")
    # print(changed_ann_file)
    changed_ann_file= changed_ann_file.replace('bruk', "bruk_changed")
    shutil.copyfile(ann_file, changed_ann_file)

    with open(txt_file, "r", encoding="utf-8") as txt_f:
        text = txt_f.read()

        new_text = []

        for x in text.split("\n"):
            if x in orig_and_changed_dict.keys():
                new_text.append(orig_and_changed_dict[x])
                # print("here:", x, " ----to----", orig_and_changed_dict[x])
            else:
                new_text.append(x)
    # print("ng" in txt_file)
    changed_txt_file = txt_file.replace('ng', "ng_changed")
    changed_txt_file = changed_txt_file.replace('bruk', "bruk_changed")
    # print(changed_txt_file)
    file_and_text_dict[changed_txt_file] = "\n".join(new_text)
        

    return file_and_text_dict

In [106]:
res = change_orig_text_to_gend_swapped(ng_test[:2])

for file_path, content in res.items():
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)

    print(f"File created: {file_path}")

/Users/linndfors/study/diploma/ner_for_fem/data/ng_changed/97c7987b8f9c.ann
here: Співвласником цієї компанії у свою чергу з часткою в 75% є Іван Плачков, голова наглядової ради ПАТ «Київенерго», яке і проводило тендер.  ----to---- Співвласницею цієї компанії у свою чергу з часткою в 75% є Ірина Плачкова, голова наглядової ради ПАТ «Київенерго», яке і проводило тендер.
here: Одесит Іван Плачков почав працювати в компанії понад 20 років тому, з середини 90-х займає у ній керівні посади, у 1995-99 вперше стає головою правління «Київенерго».  ----to---- Одеситка Ірина Плачкова почала працювати в компанії понад 20 років тому, з середини 90-х займає у ній керівні посади, у 1995-99 вперше стає головою правління «Київенерго».
here: У 1999 очолив Міністерство енергетики України, потім – знову «Київенерго» у 2000-2005 р. р. ; у 2005-2006 міністр палива та енергетики України ; багаторазовий депутат Київської міськради, з 2008 – у фракції Кличка.  ----to---- У 1999 очолила Міністерство енергетики

In [225]:
def dict_from_ann_files(directory):
    # create dictionary out of annotations
    ann_files = find_ann_files(directory)
    ann_files_content = {}

    for ann_file in ann_files:
        with open(ann_file, "r", encoding="utf-8") as ann_f:
            rows = []
            for ann_l in ann_f:
                ind, label, start, end, ent = ann_l.split("\t")

                try:
                    rows.append({"ind": ind, "label": label, "start": int(start), "end": int(end), "ent": ent})
                except Exception as e:
                    continue
        ann_files_content[ann_file] = rows

    return ann_files_content

In [226]:
import spacy
from difflib import SequenceMatcher


def tokenize_with_indices(text):
    """Tokenizes text and returns a list of (token, start, end)."""
    doc = nlp(text)
    return [(token.text, token.idx, token.idx + len(token)) for token in doc]

In [341]:
def create_new_ann_file(orig_file_name, changed_file_name, changed_text, ann_files):
    with open (orig_file_name, 'r') as file:
        original_text = file.read()

    # changed_file_name = orig_file_name.replace("ng", "ng_changed")
    old_ann_file = changed_file_name.replace(".txt", ".ann")
    original_annotations = ann_files[old_ann_file]

    original_tokens = tokenize_with_indices(original_text)
    changed_tokens = tokenize_with_indices(changed_text)

    dict_for_changed_ann = {}
    offset = 0
    for idx, (orig_token, changed_token) in enumerate(zip(original_tokens, changed_tokens)):
        if orig_token != changed_token:
            print(f"{orig_token} ---- {changed_token}")
            # print(changed_token[1], orig_token[1])
            
            # if orig_token[1] == 1605:
            #     print("-------HERE-------")
            #     print(orig_token, "-----------", changed_token)        
            for annotation in original_annotations:
                if annotation['start'] + offset == changed_token[1]:
                    print("found relevant entity with the same start:", annotation['ent'])
                    # знайшли ентіті з анотацій
                    if changed_token[2] != annotation['end'] + offset:
                        # це ентіті змінилось
                        print("this token changed its end index")
                        if changed_token[2] == (annotation['end'] + (len(changed_token[0]) - len(orig_token[0]))):
                            # просто змінилось саме слово
                            print("---edit old ent (1 ent = 1 token):", changed_token[0], (changed_token[1], changed_token[2]))
                            new_ent = changed_token[0] + "\n"
                            dict_for_changed_ann[annotation['ind']] = (new_ent, (changed_token[1], changed_token[2]))
                        else:
                            print ("---this token is a part of more than 2 tokens entity")
                            counter = 0
                            difference = 0
                            while idx + counter + 1 < len(original_tokens):
                                difference += (len(changed_tokens[idx + counter][0]) - len(original_tokens[idx + counter][0]))
                                # print("difference:", difference)
                                # print("current end:", changed_tokens[idx + counter][2])
                                if changed_tokens[idx + counter][2] == annotation['end'] + offset + difference:
                                    ind_change = idx + counter + 1
                                    new_ent_list = changed_tokens[idx:ind_change]
                                    new_ent = " ".join(ent[0] for ent in new_ent_list) + "\n"
                                    new_ind = (changed_tokens[idx][1], changed_tokens[idx + counter][2])
                                    dict_for_changed_ann[annotation['ind']] = (new_ent, new_ind)
                                    print("new ent:", (new_ent, new_ind))
                                    break
                                counter += 1
                    else:
                        # це ентіті не змінилось, а лише посунулось
                        # print("offset:", offset)
                        print("update old ent (the same ent, but moved by offset):", annotation['ent'], (annotation['start'] + offset, annotation['end'] + offset))
                        dict_for_changed_ann[annotation['ind']] = (annotation['ent'], (annotation['start'] + offset, annotation['end'] + offset))
            offset += len(changed_token[0]) - len(orig_token[0])
            print("offset:", offset)



    return dict_for_changed_ann

In [ ]:
ann_files = dict_from_ann_files("/Users/linndfors/study/diploma/ner_for_fem/data/ng_changed")
new_ann_dict = {}

for old_file in ng_test[:2]:
    ann_file = old_file.replace(".txt", ".ann")
    ann_file = ann_file.replace("ng", "ng_changed")
    new_file_new_text_dict = change_orig_text_to_gend_swapped(old_file)

    for file_path, content in new_file_new_text_dict.items():
        # os.makedirs(os.path.dirname(file_path), exist_ok=True)
        
        # with open(file_path, 'w', encoding='utf-8') as file:
        #     file.write(content)

        # print(f"File created: {file_path}")
        print("ann file:", ann_file)
        dict_for_changed_ann = create_new_ann_file(old_file, file_path, content, ann_files)

        # print("dict:", dict_for_changed_ann)

        for x in ann_files[ann_file]:
            # print(x)
            if x['ind'] in dict_for_changed_ann.keys():
                # new_ann_dict[ann_file]
                x['end'] = dict_for_changed_ann[x['ind']][1][1]
                x['start'] = dict_for_changed_ann[x['ind']][1][0]
                new_ent = dict_for_changed_ann[x['ind']][0]
                x['ent'] = new_ent
                print(x)

# orig_file_name = '/Users/linndfors/study/diploma/ner_for_fem/data/ng/605667093f7b.txt'

ann file: /Users/linndfors/study/diploma/ner_for_fem/data/ng_changed/97c7987b8f9c.ann
('Співвласником', 599, 612) ---- ('Співвласницею', 599, 612)
offset: 0
('Іван', 658, 662) ---- ('Ірина', 658, 663)
found relevant entity with the same start: Іван Плачков

this token changed its end index
---this token is a part of more than 2 tokens entity
new ent: ('Ірина Плачкова\n', (658, 672))
offset: 1
('Плачков', 663, 670) ---- ('Плачкова', 664, 672)
offset: 2
(',', 670, 671) ---- (',', 672, 673)
offset: 2
('голова', 672, 678) ---- ('голова', 674, 680)
found relevant entity with the same start: голова наглядової ради

this token changed its end index
---this token is a part of more than 2 tokens entity
new ent: ('голова наглядової ради\n', (674, 696))
offset: 2
('наглядової', 679, 689) ---- ('наглядової', 681, 691)
offset: 2
('ради', 690, 694) ---- ('ради', 692, 696)
offset: 2
('ПАТ', 695, 698) ---- ('ПАТ', 697, 700)
found relevant entity with the same start: ПАТ «Київенерго»

this token change

In [ ]:

def write_ann_files_from_dict(ann_files_content):
    for ann_file, annotations in ann_files_content.items():
        print(ann_file)
        ann_file = ann_file.replace(".ann", "_changed.ann")
        with open(ann_file, "w", encoding="utf-8") as f:
            for annotation in annotations:
                line = f"{annotation['ind']}\t{annotation['label']}\t{annotation['start']}\t{annotation['end']}\t{annotation['ent']}"
                f.write(line)

In [344]:
write_ann_files_from_dict(ann_files)

/Users/linndfors/study/diploma/ner_for_fem/data/ng_changed/605667093f7b.ann
/Users/linndfors/study/diploma/ner_for_fem/data/ng_changed/97c7987b8f9c.ann
